<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [11]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [3]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [4]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/91.0.4472.124 Safari/537.36"
}

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [12]:
from bs4 import BeautifulSoup

# Ezt az URL-t most már nem fogjuk lekérni Pythonból, csak infotartalék:
static_url = "https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches"

# A HTML-t a feltöltött fájlból olvassuk be
with open("falcon9_wiki.html", "r", encoding="utf-8") as f:
    html = f.read()

# BeautifulSoup objektum
soup = BeautifulSoup(html, "html.parser")


Create a `BeautifulSoup` object from the HTML `response`


In [13]:
soup.title


<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

Print the page title to verify if the `BeautifulSoup` object was created properly 


In [14]:
print(soup.title)          # teljes <title> tag
print(soup.title.string)   # csak a szöveg, ha ezt szeretnéd


<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>
List of Falcon 9 and Falcon Heavy launches - Wikipedia


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [23]:
# Keressük meg az összes táblát az oldalon
html_tables = soup.find_all("table")

print("Talált táblák száma:", len(html_tables))


Talált táblák száma: 8


Starting from the third table is our target table contains the actual launch records.


In [24]:
first_launch_table = html_tables[2]
print(first_launch_table)


<table 100%;"="" class='3D"wikitable' id='3D"F9-xx=' sticky-header"="" style='3D"width:' x"="">
<tbody><tr>
<th scope='3D"col"'>Date and time ()
</th>
<th scope='3D"col"'>
</th>
<th scope='3D"col"'>Launch site
</th>
<th scope='3D"col"'>Payload
</th>
<th scope='3D"col"'>Orbit
</th>
<th scope='3D"col"'>Customer
</th></tr>
<tr>
<td rowspan='3D"2"'>December 7, 2025<br/>16:13
</td>
<td><a 5"="" 9="" block="" con="" href='3D"https://en.wikipedia.org/wiki/Falcon_9_Block_5"' title='3D"Fal='>F9<span class='3D"nowrap"'> </span>B5</a><br/><a 9="" boosters"="" en.wikipedia.org="" falcon="" first-stage="" href='3D"h=' list_of_falcon_9_first-stage_boosters#b1088"="" of="" t='itle=3D"List' ttps:="" wiki="">B1088=E2=80=9112</a>
</td>
<td><a base"="" force="" href='3D"https://en.wikipedia.org/wiki/Vandenberg_Space_Force_Base"' space="" t='itle=3D"Vandenberg'>Vandenberg</a>, <a 4"="" complex="" href='3D"https://en.=' launch="" s="pace" title='3D"Vandenberg' vandenberg_space_launch_complex_4"="" wiki="" 

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [27]:
column_names = [
    'Date and time ( )',
    'Version, booster',
    'Launch site',
    'Payload',
    'Orbit',
    'Customer'
]

print(column_names)



['Date and time ( )', 'Version, booster', 'Launch site', 'Payload', 'Orbit', 'Customer']


Check the extracted column names


In [28]:
print(column_names)

['Date and time ( )', 'Version, booster', 'Launch site', 'Payload', 'Orbit', 'Customer']


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [29]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [32]:
extracted_row = 0

# Végigmegyünk az összes indítási táblán
for table_number, table in enumerate(
        soup.find_all('table', 'wikitable plainrowheaders collapsible')):

    # Soronként bejárjuk a táblát
    for row in table.find_all('tr'):

        # A valódi indítási sorokat a sorfejlécben lévő sorszám jelzi
        th = row.find('th')
        if not th:
            continue

        flight_str = th.get_text(strip=True)


        if not flight_str.isdigit():
            continue

        extracted_row += 1
        flight_no = int(flight_str)

        cells = row.find_all('td')
        if len(cells) < 7:   # védekezés a félbehagyott sorok ellen
            continue

        dt = date_time(cells[0])         # helper függvény a notebook elejéről
        launch_dict['Flight No.'].append(flight_no)
        launch_dict['Date'].append(dt[0].strip())
        launch_dict['Time'].append(dt[1].strip())

        v_booster = booster_version(cells[1])   # helper
        launch_dict['Version Booster'].append(v_booster)

        launch_site = cells[2].get_text(" ", strip=True)
        launch_dict['Launch site'].append(launch_site)

        payload = cells[3].get_text(" ", strip=True)
        launch_dict['Payload'].append(payload)

        mass = get_mass(cells[3])       # helper
        launch_dict['Payload mass'].append(mass)

        orbit = cells[4].get_text(" ", strip=True)
        launch_dict['Orbit'].append(orbit)

        customer = cells[5].get_text(" ", strip=True)
        launch_dict['Customer'].append(customer)

        outcome = cells[6].get_text(" ", strip=True)
        launch_dict['Launch outcome'].append(outcome)

        booster_landing = landing_status(cells[7])   # helper
        launch_dict['Booster landing'].append(booster_landing)


After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [31]:
df = pd.DataFrame(launch_dict)
df.head()


,"Version, booster",Launch site,Payload,Orbit,Customer,Flight No.,Payload mass,Launch outcome,Version Booster,Booster landing,Date,Time


We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
